# Day 06 of 14 Days Databricks Challenge


In [0]:
raw = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/*.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql import functions as F

In [0]:
raw.withColumn("ingestion_ts", F.current_timestamp()) \
.write.format("delta") \
.mode("overwrite") \
.save("/Volumes/workspace/ecommerce/ecommerce_data/bronze/events") 


In [0]:
def log(msg):
    print(f"🔹 {msg}")

In [0]:
log("Verifying BRONZE layer")
spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/bronze/events").show(5, truncate=False)


In [0]:
log("Starting SILVER processing")

bronze = spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/bronze/events")

log(f"Bronze records loaded: {bronze.count():,}")

In [0]:
silver = bronze.filter(F.col("price") > 0) \
    .filter(F.col("price") < 10000)

log(f"Records after price validation: {silver.count():,}")


In [0]:
before_dedup = silver.count()

silver = silver.dropDuplicates(["user_session", "event_time"])

after_dedup = silver.count()

log(f"Duplicates removed: {before_dedup - after_dedup:,}")


In [0]:
silver = silver.withColumn("event_date", F.to_date("event_time")) \
    .withColumn(
        "price_tier",
        F.when(F.col("price") < 10, "budget")
         .when(F.col("price") < 50, "mid")
         .otherwise("premium")
    )

log("Derived event_date and price_tier columns")


In [0]:
silver.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/ecommerce/ecommerce_data/silver/events")

log("SILVER write completed")


In [0]:
log("Verifying SILVER layer")
spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/silver/events").show(5, truncate=False)


In [0]:
log("Starting GOLD aggregation")

silver = spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/silver/events")

log(f"Silver records loaded: {silver.count():,}")


In [0]:
product_perf = silver.groupBy("product_id", "brand") \
    .agg(
        F.countDistinct(
            F.when(F.col("event_type") == "view", "user_id")
        ).alias("views"),
        F.countDistinct(
            F.when(F.col("event_type") == "purchase", "user_id")
        ).alias("purchases"),
        F.sum(
            F.when(F.col("event_type") == "purchase", "price")
        ).alias("revenue")
    )


In [0]:
product_perf = product_perf.withColumn(
    "conversion_rate",
    F.when(F.col("views") > 0,
           F.col("purchases") / F.col("views") * 100
    ).otherwise(0)
)

log("Calculated product-level metrics")


In [0]:
log(f"Gold records (products): {product_perf.count():,}")
product_perf.show(5, truncate=False)


In [0]:
from pyspark.sql.functions import col

silver = bronze.withColumn(
    "price_clean",
    F.expr("try_cast(price as double)")
)


In [0]:
silver = silver.filter(col("price_clean").isNotNull()) \
               .filter(col("price_clean") > 0) \
               .filter(col("price_clean") < 10000)


In [0]:
silver = silver.drop("price") \
               .withColumnRenamed("price_clean", "price")


In [0]:
bad_prices = bronze.filter(
    F.expr("try_cast(price as double)").isNull()
).count()

print(f"⚠️ Bad price records removed: {bad_prices:,}")


In [0]:
product_perf = silver.groupBy("product_id", "brand") \
    .agg(
        F.countDistinct(
            F.when(F.col("event_type") == "view", "user_id")
        ).alias("views"),
        F.countDistinct(
            F.when(F.col("event_type") == "purchase", "user_id")
        ).alias("purchases"),
        F.sum(
            F.when(F.col("event_type") == "purchase", F.col("price"))
        ).alias("revenue")
    )


In [0]:
product_perf = product_perf.withColumn(
    "conversion_rate",
    F.when(F.col("views") > 0,
           F.col("purchases") / F.col("views") * 100
    ).otherwise(0)
)

log("Calculated product-level metrics")


In [0]:
log(f"Gold records (products): {product_perf.count():,}")
product_perf.show(5, truncate=False)

In [0]:
product_perf.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/ecommerce/ecommerce_data/gold/products")

log("GOLD write completed")


In [0]:
log("Medallion pipeline completed successfully (Bronze → Silver → Gold)")
